In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

open MathNet.Numerics.LinearAlgebra

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package MathNet.Numerics.FSharp version 4.15.0

## 1.1 Постановка задачи
1) Решить две системы (одну с “точной” правой частью, вторую с изменённой)  
2) Найти число обусловленности исходной матрицы  
3) Посчитать фактическую погрешность и оценку погрешности

### Вариант 9

In [ ]:
let a = matrix [
    [-401.64; 200.12]
    [1200.72; -601.76]
]

let bAccurate = vector [200.; -600.]
let bAltered = vector [199.; -601.]

let deltaA = DenseMatrix.zero<float> 2 2
let deltaB = bAltered - bAccurate

### Решение

In [ ]:
let conditionNumber (matrix: Matrix<'a>) = 
    matrix.Inverse().L2Norm() * matrix.L2Norm()

let solutionAccurate = a.Solve bAccurate
let solutionAltered = a.Solve bAltered

let condA = conditionNumber a

let errorFact = (solutionAltered - solutionAccurate).L2Norm() / solutionAccurate.L2Norm()
let errorEstimation = 
    condA / (1. - condA * deltaA.L2Norm() / a.L2Norm()) * 
    (deltaB.L2Norm() / bAccurate.L2Norm() + deltaA.L2Norm() / a.L2Norm())

// printfn "Решение с точной правой частью: %A" solutionAccurate 
// printfn "Решение с изменённой правой частью: %A" solutionAltered
// printfn "Число обусловенности: %A" condA
// printfn "Фактическая погрешность: %A" errorFact
// printfn "Оценка погрешности: %A" errorEstimation

## 1.2 Постановка задачи
1) Реализовать метод Гаусса с выбором главного элемента для решения СЛАУ  
2) Реализовать метод Жордана для нахождения обратной матрицы  
3) Реализовать метод LU-разложения для нахождения определителя матрицы

### Вариант 9

In [ ]:
let a = matrix [
    [12.78572; 1.534675; -3.947418]
    [1.534675; 9.709232; 0.918435]
    [-3.947418; 0.918435; 7.703946]
]

let b = vector [9.60565; 7.30777; 4.21575]

### Метод Гаусса в выбором главных компонент

In [ ]:
let solveGauss (matrix: Matrix<float>) (freeVector: Vector<float>) = 
    let n = matrix.RowCount
    let extendedMatrix = matrix |> Matrix.appendCol freeVector
    for k = 0 to n - 1 do
        let temp = extendedMatrix.[k, k]
        if abs temp > 1e-16 then
            for j = k + 1 to n do
                extendedMatrix.[k, j] <- extendedMatrix.[k, j] / temp
        for i = k + 1 to n - 1 do
            let temp = extendedMatrix.[i, k]
            for j = k to n do
                extendedMatrix.[i, j] <- extendedMatrix.[i, j] - extendedMatrix.[k, j] * temp 

    let solution = DenseVector.zero<float> n
    for i = n - 1 downto 0 do
        let mutable sum = 0.
        for j = i + 1 to n - 1 do
            sum <- sum + extendedMatrix.[i, j] * solution.[j]
        solution.[i] <- extendedMatrix.[i, n] - sum

    solution   

In [ ]:
// let solution = solveGauss a b
// printfn "Решение системы методом Гаусса: %A" solution
// printfn "Компоненты вектора невязки R = b - Ax: %s" <| (b - a * solution).ToString("N18")

### Метод Жордана для нахождения обратной матрицы

In [ ]:
let inverseJordan (matrix: Matrix<float>) = 
    let n = matrix.RowCount
    let extendedMatrix = matrix.Append <| DenseMatrix.identity n
    for k = 0 to n - 1 do
        let temp = extendedMatrix.[k, k]
        if abs temp > 1e-16 then
            for j = k + 1 to 2 * n - 1 do
                extendedMatrix.[k, j] <- extendedMatrix.[k, j] / temp
        for i in [0 .. k - 1] @ [k + 1 .. n - 1] do
            let temp = extendedMatrix.[i, k]
            for j = k to 2 * n - 1 do
                extendedMatrix.[i, j] <- extendedMatrix.[i, j] - extendedMatrix.[k, j] * temp   
    
    extendedMatrix.[*, n .. 2 * n - 1]

In [ ]:
// let inversed = inverseJordan a
// printfn "Обратная матрица: %A" <| a.Inverse()
// printfn "Обратная матрица методом Жордана: %A" inversed
// printfn "Компоненты матрицы невязки: %s" <| (a.Inverse() - inversed).ToString("N18")


### Метод LU-разложения для нахождения определителя матрицы

In [ ]:
let detLU (matrix: Matrix<float>) = 
    let n = matrix.RowCount
    let l = DenseMatrix.zero<float> n n
    let u = DenseMatrix.zero<float> n n

    for i = 0 to n - 1 do
        for j = i to n - 1 do
            let mutable sum = 0.
            for k = 0 to i - 1 do
                sum <- sum + l.[j, k] * u.[k, i]
            l.[j, i] <- matrix.[j, i] - sum

        for j = i to n - 1 do
            let mutable sum = 0.
            for k = 0 to i - 1 do
                sum <- sum + l.[i, k] * u.[k, j]
            u.[i, j] <- (matrix.[i, j] - sum) / l.[i, i]

    l.Diagonal() |> Seq.reduce (*)


In [ ]:
printfn "Определитель матрицы: %A" <| a.Determinant()
printfn "Определитель матрицы с помощью LU-разложения: %A" <| detLU a

Определитель матрицы: 765.0165055
Определитель матрицы с помощью LU-разложения: 765.0165055
